In [2]:
import scipy.io as sio
import numpy as np
import h5py
import os
import random
import argparse

import torch
from torch import double, optim, var
import torch.nn as nn

In [3]:
from modelDesign import *

In [4]:
  def set_quantization(model, is_quantization = True):
    try:
        model.encoder.quantization = is_quantization
        model.decoder.quantization = is_quantization
    except:
        model.module.encoder.quantization = is_quantization
        model.module.decoder.quantization = is_quantization
    return model

In [5]:
model = AutoEncoder(512)
model = set_quantization(model, True)

In [6]:
model.encoder.load_state_dict(torch.load('./modelSubmit' + '/encoder.pth.tar')['state_dict'])
model.decoder.load_state_dict(torch.load('./modelSubmit' + '/decoder.pth.tar')['state_dict'])
model = model.cuda()

In [7]:
mat = sio.loadmat('./channelData' + '/H_4T4R.mat')

In [8]:
data = mat['H_4T4R']  # shape=(320000, 1024)
data = data.astype('float32')
data = np.reshape(data, [len(data), 24, 16, 2])
# data = torch.Tensor(data)
# data = data.cuda()

In [9]:
data = DatasetFolder(data)
data_all = torch.utils.data.DataLoader(
        data, batch_size=256, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

In [10]:
bit = torch.Tensor()
bit = bit.cuda()
for i, input in enumerate(data_all):
    input = input.cuda()
    bit = torch.cat((model.encoder(input), bit), 0)

RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 23.70 GiB total capacity; 21.88 GiB already allocated; 68.25 MiB free; 21.93 GiB reserved in total by PyTorch)

In [28]:
bit_mean = torch.mean(bit)